In [0]:
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import np_utils

# 연습문제: 동요 만들기

In [0]:
# 코드 사전 정의(동요)

code2idx = {'c4':0, 'd4':1, 'e4':2, 'f4':3, 'g4':4, 'a4':5, 'b4':6,
            'c8':7, 'd8':8, 'e8':9, 'f8':10, 'g8':11, 'a8':12, 'b8':13}

idx2code = {0:'c4', 1:'d4', 2:'e4', 3:'f4', 4:'g4', 5:'a4', 6:'b4',
            7:'c8', 8:'d8', 9:'e8', 10:'f8', 11:'g8', 12:'a8', 13:'b8'}

# 시퀀스 데이터 정의

seq = ['g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'd8', 'e8', 'f8', 'g8', 'g8', 'g4',
       'g8', 'e8', 'e8', 'e8', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4',
       'd8', 'd8', 'd8', 'd8', 'd8', 'e8', 'f4', 'e8', 'e8', 'e8', 'e8', 'e8', 'f8', 'g4',
       'g8', 'e8', 'e4', 'f8', 'd8', 'd4', 'c8', 'e8', 'g8', 'g8', 'e8', 'e8', 'e4']

## 데이터 전처리

In [31]:
# 순차적으로 입력된 데이터를 window size로 잘라주는 함수

def seq2dataset(seq, window_size):
  dataset = []
  for i in range(len(seq)-window_size):    # len(seq) = 54    # 0~49    # 49, 50, 51, 52번 인덱스로 53번 인덱스를 예측하는게 마지막이므로. (갯수가 54개이므로 인덱스는 53까지)
    subset = seq[i:(i+window_size+1)]    # 입력데이터와 출력데이터를 함께 subset에 넣는다
    dataset.append([code2idx[c] for c in subset])
  return np.array(dataset)

dataset = seq2dataset(seq, window_size=4)
dataset

array([[11,  9,  2, 10,  8],
       [ 9,  2, 10,  8,  1],
       [ 2, 10,  8,  1,  7],
       [10,  8,  1,  7,  8],
       [ 8,  1,  7,  8,  9],
       [ 1,  7,  8,  9, 10],
       [ 7,  8,  9, 10, 11],
       [ 8,  9, 10, 11, 11],
       [ 9, 10, 11, 11,  4],
       [10, 11, 11,  4, 11],
       [11, 11,  4, 11,  9],
       [11,  4, 11,  9,  9],
       [ 4, 11,  9,  9,  9],
       [11,  9,  9,  9, 10],
       [ 9,  9,  9, 10,  8],
       [ 9,  9, 10,  8,  1],
       [ 9, 10,  8,  1,  7],
       [10,  8,  1,  7,  9],
       [ 8,  1,  7,  9, 11],
       [ 1,  7,  9, 11, 11],
       [ 7,  9, 11, 11,  9],
       [ 9, 11, 11,  9,  9],
       [11, 11,  9,  9,  2],
       [11,  9,  9,  2,  8],
       [ 9,  9,  2,  8,  8],
       [ 9,  2,  8,  8,  8],
       [ 2,  8,  8,  8,  8],
       [ 8,  8,  8,  8,  8],
       [ 8,  8,  8,  8,  9],
       [ 8,  8,  8,  9,  3],
       [ 8,  8,  9,  3,  9],
       [ 8,  9,  3,  9,  9],
       [ 9,  3,  9,  9,  9],
       [ 3,  9,  9,  9,  9],
       [ 9,  9

In [32]:
dataset.shape

(50, 5)

In [0]:
# 입출력 변수 분리

xtrain = dataset[:, :4]
ytrain = dataset[:, 4]

# 정규화

max_idx_value = 13
xtrain = xtrain/float(max_idx_value)

# 원핫인코딩

ytrain = np_utils.to_categorical(ytrain)
ytrain.shape    # (50, 12)    # 12 = 동요에서 사용된 코드의 개수
one_hot_vec_size = ytrain.shape[1]    # 12

## 모델 생성

In [0]:
# 다층 퍼셉트론 모델링 (LSTM 안씀)

model = Sequential()
model.add(Dense(128, input_dim=4, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(one_hot_vec_size, activation="softmax"))

In [0]:
# 학습 알고리즘 설정

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [0]:
class LossHistory(keras.callbacks.Callback):
  def init(self):
    self.losses = []
  def on_epoch_end(self, batch, logs={}):    # 에폭이 끝났을 때 시스템에 의해 자동 호출되는 함수
    self.losses.append(logs.get("loss"))

In [0]:
history = LossHistory()
history.init()

In [39]:
num_epochs = 1000

model.fit(xtrain, ytrain, epochs=num_epochs, batch_size=1, verbose=2, callbacks=[history])

Epoch 1/1000
 - 0s - loss: 1.6730 - acc: 0.3800
Epoch 2/1000
 - 0s - loss: 1.6347 - acc: 0.4000
Epoch 3/1000
 - 0s - loss: 1.6352 - acc: 0.4000
Epoch 4/1000
 - 0s - loss: 1.6063 - acc: 0.4000
Epoch 5/1000
 - 0s - loss: 1.6226 - acc: 0.3800
Epoch 6/1000
 - 0s - loss: 1.5997 - acc: 0.4200
Epoch 7/1000
 - 0s - loss: 1.5952 - acc: 0.4200
Epoch 8/1000
 - 0s - loss: 1.5713 - acc: 0.4200
Epoch 9/1000
 - 0s - loss: 1.5800 - acc: 0.3800
Epoch 10/1000
 - 0s - loss: 1.5583 - acc: 0.4800
Epoch 11/1000
 - 0s - loss: 1.5505 - acc: 0.4400
Epoch 12/1000
 - 0s - loss: 1.5361 - acc: 0.4800
Epoch 13/1000
 - 0s - loss: 1.5323 - acc: 0.4400
Epoch 14/1000
 - 0s - loss: 1.5425 - acc: 0.4600
Epoch 15/1000
 - 0s - loss: 1.5011 - acc: 0.4400
Epoch 16/1000
 - 0s - loss: 1.4981 - acc: 0.4800
Epoch 17/1000
 - 0s - loss: 1.5107 - acc: 0.5000
Epoch 18/1000
 - 0s - loss: 1.4958 - acc: 0.4400
Epoch 19/1000
 - 0s - loss: 1.4723 - acc: 0.4000
Epoch 20/1000
 - 0s - loss: 1.4885 - acc: 0.4800
Epoch 21/1000
 - 0s - loss: 1

In [43]:
scores = model.evaluate(xtrain,ytrain, batch_size=1)
print("%.2f%%" % (scores[1]*100))

50/50 [==============================] - 0s 1ms/step
92.00%


## 예측

In [45]:
seq_out = ['g8', 'e8', 'e4', 'f8']
pred_out = model.predict(xtrain, batch_size=1)
pred_out

array([[1.83470703e-15, 0.00000000e+00, 1.13786367e-20, 1.49156149e-14,
        3.70794734e-16, 9.81501122e-16, 1.44519597e-15, 1.22842272e-23,
        9.97212231e-01, 2.78775394e-03, 1.10624999e-11, 6.98963263e-23],
       [4.45740137e-17, 9.99857903e-01, 5.19287605e-05, 1.78125986e-27,
        4.50677771e-14, 1.75673260e-17, 2.01081056e-17, 3.92913996e-19,
        5.90002446e-06, 8.42136506e-05, 2.73380334e-16, 5.63930647e-17],
       [7.39598468e-27, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.40377832e-28, 1.28405322e-27, 9.99825418e-01,
        8.39781485e-17, 1.19063907e-05, 1.62697703e-04, 8.19512253e-23],
       [3.72287352e-13, 4.29623155e-31, 2.91191915e-09, 3.28053014e-23,
        5.51676684e-23, 2.66723357e-13, 3.41510864e-13, 1.50459900e-09,
        3.61575395e-01, 6.38424575e-01, 1.54157043e-08, 2.26999540e-27],
       [9.22158790e-12, 2.08332906e-10, 8.01318707e-19, 8.47398818e-10,
        5.00108331e-12, 5.04328870e-12, 4.18221360e-12, 8.37

In [49]:
np.argmax(pred_out, 1)

array([ 8,  1,  7,  9,  9, 10, 11, 11,  4, 11,  9,  9,  9, 10,  8,  1,  7,
        9, 11, 11,  9,  9,  2,  8,  8,  8,  8,  9,  9,  3,  9,  9,  9,  9,
       10, 10,  8, 11,  9,  2, 10,  8,  1,  7,  9, 11, 11,  9,  9,  2])

In [50]:
for i in range(50):    # 50: 예측개수
  idx = np.argmax(pred_out[i])
  seq_out.append(idx2code[idx])

seq_out

['g8',
 'e8',
 'e4',
 'f8',
 'd8',
 'd4',
 'c8',
 'e8',
 'e8',
 'f8',
 'g8',
 'g8',
 'g4',
 'g8',
 'e8',
 'e8',
 'e8',
 'f8',
 'd8',
 'd4',
 'c8',
 'e8',
 'g8',
 'g8',
 'e8',
 'e8',
 'e4',
 'd8',
 'd8',
 'd8',
 'd8',
 'e8',
 'e8',
 'f4',
 'e8',
 'e8',
 'e8',
 'e8',
 'f8',
 'f8',
 'd8',
 'g8',
 'e8',
 'e4',
 'f8',
 'd8',
 'd4',
 'c8',
 'e8',
 'g8',
 'g8',
 'e8',
 'e8',
 'e4']